# Start

In [1]:
import importlib
import os
# import pandas as pd
# import json
# from datetime import datetime
from pathlib import Path
# from io import StringIO

from langchain_core.tools import Tool, StructuredTool
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage, SystemMessage

from bsm_multi_agents.graph.state import WorkflowState

In [2]:
# from bsm_multi_agents.agents import utils
# importlib.reload(utils)
# from bsm_multi_agents.agents.utils import get_tool_result_from_messages,print_resp

In [3]:
cwd = Path.cwd()
project_path = cwd.parent.parent
project_path

PosixPath('/Users/yifanli/Github/model_doc_automation')

In [4]:
import asyncio
from mcp.client.stdio import stdio_client, StdioServerParameters
from mcp.client.session import ClientSession


async def call_mcp_tool_test(
    tool_name: str, 
    server_path: str, 
    arguments: dict
):
    """
    直接通过 MCP 协议调用某个 tool_name，并把结果对象返回。
    """
    server_params = StdioServerParameters(
        command="python",
        args=[str(server_path)],  # 用上面定义的 server_path
    )

    async with stdio_client(server_params) as (read, write):
        async with ClientSession(read, write) as session:
            # 初始化 MCP 会话（握手）
            await session.initialize()

            # 可选：先看看这个 server 暴露了哪些工具
            tools = await session.list_tools()
            print("Tools exposed by MCP server:")
            for t in tools.tools:
                print("  -", t.name)

            # 调用具体工具
            res = await session.call_tool(
                tool_name,
                arguments=arguments,
            )
            return res

# Graph

In [6]:
# from langgraph.graph import StateGraph, END, START

# from bsm_multi_agents.graph import state
# importlib.reload(state)
# from bsm_multi_agents.graph.state import WorkflowState
# from bsm_multi_agents.agents.data_loader_agent import data_loader_node
# from bsm_multi_agents.agents.calculator_agent import calculator_node
# from bsm_multi_agents.agents.validator_agent import validator_node

# graph = StateGraph(WorkflowState)
# graph.add_node("data_loader", data_loader_node)
# graph.add_node("calculator", calculator_node)
# graph.add_node("validator", validator_node)

# graph.add_edge(START, "data_loader")
# graph.add_edge("data_loader", "calculator")
# graph.add_edge("calculator", "validator")
# graph.add_edge("validator", END)

# app = graph.compile()

In [7]:
# file_path = os.path.join(project_path, "data/input/dummy_options.csv")
# init_state: WorkflowState = {
#     "csv_file_path": str(file_path),
#     "remaining_steps": 10,
#     "messages": [HumanMessage(content=f"Load CSV from: {file_path}")],
# }

# final_state = app.invoke(
#     init_state,
#     config={"configurable": {"thread_id": "run-1"}}
# )

In [8]:
# final_state

# Calculator

## MCP Tool Level

### calculate_bsm_to_file

In [ ]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [12]:
from bsm_multi_agents.mcp import tool_calculator
importlib.reload(tool_calculator)
from bsm_multi_agents.mcp.tool_calculator import calculate_bsm_to_file

In [ ]:
calculate_bsm_to_file(csv_file_path, output_dir)

'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv'

### calculate_greeks_to_file

In [5]:
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [6]:
from bsm_multi_agents.mcp import tool_calculator
importlib.reload(tool_calculator)
from bsm_multi_agents.mcp.tool_calculator import calculate_greeks_to_file

In [7]:
calculate_greeks_to_file(csv_file_path, output_dir)

'/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results.csv'

## MCP Server Level

## Call MCP Level

### call_mcp_tool_async

In [5]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [6]:
from bsm_multi_agents.agents import mcp_client
importlib.reload(mcp_client)
from bsm_multi_agents.agents.mcp_client import call_mcp_tool_async

In [7]:
res = await call_mcp_tool_async(
    tool_name="calculate_bsm_to_file",
    server_script_path=server_path,
    arguments={
        "input_path": csv_file_path,
        "output_dir": output_dir,
    },
)
res

CallToolResult(meta=None, content=[TextContent(type='text', text='/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv', annotations=None, meta=None)], structuredContent={'result': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv'}, isError=False)

### call_mcp_tool

### run_in_new_loop

### create_mcp_state_tool_wrapper

In [5]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [6]:
current_state = {
    "csv_file_path": csv_file_path,
    "output_dir": output_dir
}

In [7]:
from bsm_multi_agents.agents import mcp_client
importlib.reload(mcp_client)
from bsm_multi_agents.agents.mcp_client import create_mcp_state_tool_wrapper

In [8]:
bsm_state_tool = create_mcp_state_tool_wrapper(
    mcp_tool_name="calculate_bsm_to_file",
    server_script_path=server_path,
    input_arg_map={
        "csv_file_path": "input_path",
        "output_dir": "output_dir"
    },
    output_key="bsm_results_path"
)

In [9]:
result_update = bsm_state_tool(state=current_state)

In [10]:
result_update

{'bsm_results_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv'}

## Agent Level

### calculator_tool_node

#### Node Inside Level

In [18]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [23]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

from bsm_multi_agents.agents.mcp_client import create_mcp_state_tool_wrapper

In [24]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [25]:
errors = current_state.get("errors", [])
[
    "csv_file_path" not in current_state or not current_state["csv_file_path"],
    "server_path" not in current_state or not current_state["server_path"]
]

[False, False]

In [26]:
server_path = current_state["server_path"]
bsm_state_fn = create_mcp_state_tool_wrapper(
    mcp_tool_name="calculate_bsm_to_file",
    server_script_path=server_path,
    input_arg_map={
        "csv_file_path": "input_path",
        "output_dir": "output_dir",
    },
    output_key="bsm_results_path",
)

greeks_state_fn = create_mcp_state_tool_wrapper(
    mcp_tool_name="calculate_greeks_to_file",
    server_script_path=server_path,
    input_arg_map={
        "csv_file_path": "input_path",
        "output_dir": "output_dir",
    },
    output_key="greeks_results_path",
)

In [27]:
bsm_update = bsm_state_fn(state=current_state)
greeks_update = greeks_state_fn(state=current_state)

In [29]:
current_state["bsm_results_path"] = bsm_update["bsm_results_path"]
current_state["greeks_results_path"] = greeks_update["greeks_results_path"]

#### Node Level

In [30]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [32]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

from bsm_multi_agents.agents import calculator_agent
importlib.reload(calculator_agent)
from bsm_multi_agents.agents.calculator_agent import calculator_tool_node

In [33]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [37]:
update_state = calculator_tool_node(current_state=current_state)
update_state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'bsm_results_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_bsm_results.csv',
 'greeks_results_path': '/Users/yifanli/Github/model_doc_automation/data/cache/dummy_options_greeks_results.csv',
 'errors': []}

### calculator_agent_node

#### Node Inside Level

In [38]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")
prompt_path = os.path.join(project_path, "src/bsm_multi_agents/prompts/calculator_prompts.txt")

In [42]:
from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

from bsm_multi_agents.config.llm_config import get_llm
from bsm_multi_agents.prompts.loader import load_prompt

In [43]:
current_state = WorkflowState(
        csv_file_path=csv_file_path, 
        output_dir=output_dir, 
        server_path=server_path
    )

In [44]:
errors = current_state.get("errors", [])
"csv_file_path" not in current_state or not current_state["csv_file_path"]

False

In [49]:
system_prompt = """
You are a quantitative calculator agent.
You have access to tools that call an external MCP server.
Each tool reads input file paths from the shared workflow state
and writes result file paths back into the state.
Do NOT ask for raw CSV content. Always work with paths only.
You do not need to provide any arguments to the tools. Just call them with {}.
"""
user_prompt_template = load_prompt(prompt_path)
formatted_user_prompt = (
    user_prompt_template.format(csv_file_path=current_state["csv_file_path"])
    if "{csv_file_path}" in user_prompt_template
    else user_prompt_template
)
llm = get_llm()
system_msg = SystemMessage(content=system_prompt)
human_msg = HumanMessage(content=formatted_user_prompt)
ai_msg = llm.invoke([system_msg, human_msg])

In [51]:
messages = list(current_state.get("messages", []))
messages.extend([system_msg, human_msg, ai_msg])
current_state["messages"] = messages

current_state["errors"] = errors

In [52]:
current_state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='\nYou are a quantitative calculator agent.\nYou have access to tools that call an external MCP server.\nEach tool reads input file paths from the shared workflow state\nand writes result file paths back into the state.\nDo NOT ask for raw CSV content. Always work with paths only.\nYou do not need to provide any arguments to the tools. Just call them with {}.\n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Calculate Black-Scholes-Merton option prices and greeks.\n- Load the data from the /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv.\n- Use the `calculate_bsm_to_file` tool to calculate prices.\n- Use the `calculate_greeks_to_file` tool to calc

#### Node Level

In [54]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [55]:
from bsm_multi_agents.agents import calculator_agent
importlib.reload(calculator_agent)
from bsm_multi_agents.agents.calculator_agent import calculator_agent_node

from bsm_multi_agents.graph import state
importlib.reload(state)
from bsm_multi_agents.graph.state import WorkflowState

In [56]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [57]:
result_state = calculator_agent_node(current_state)

In [58]:
result_state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='\n    You are a quantitative calculator agent.\n    You have access to tools that call an external MCP server.\n    Each tool reads input file paths from the shared workflow state\n    and writes result file paths back into the state.\n    Do NOT ask for raw CSV content. Always work with paths only.\n    You do not need to provide any arguments to the tools. Just call them with {}.\n    ', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Calculate Black-Scholes-Merton option prices and greeks.\n- Load the data from the /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv.\n- Use the `calculate_bsm_to_file` tool to calculate prices.\n- Use the `calculate_

### Combined Tool and Agent Levle

In [59]:
server_path = os.path.join(project_path, "src/bsm_multi_agents/mcp/server.py")
csv_file_path = os.path.join(project_path, "data/input/dummy_options.csv")
output_dir = os.path.join(project_path, "data/cache")

In [62]:
from bsm_multi_agents.graph.state import WorkflowState
from bsm_multi_agents.agents.calculator_agent import (
    calculator_agent_node,
    calculator_tool_node,
)

In [63]:
current_state = WorkflowState(
    csv_file_path=csv_file_path, 
    output_dir=output_dir, 
    server_path=server_path
)

In [64]:
current_state = calculator_agent_node(current_state)
current_state = calculator_tool_node(current_state)

In [65]:
current_state

{'csv_file_path': '/Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv',
 'output_dir': '/Users/yifanli/Github/model_doc_automation/data/cache',
 'server_path': '/Users/yifanli/Github/model_doc_automation/src/bsm_multi_agents/mcp/server.py',
 'messages': [SystemMessage(content='\n    You are a quantitative calculator agent.\n    You have access to tools that call an external MCP server.\n    Each tool reads input file paths from the shared workflow state\n    and writes result file paths back into the state.\n    Do NOT ask for raw CSV content. Always work with paths only.\n    You do not need to provide any arguments to the tools. Just call them with {}.\n    ', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Calculate Black-Scholes-Merton option prices and greeks.\n- Load the data from the /Users/yifanli/Github/model_doc_automation/data/input/dummy_options.csv.\n- Use the `calculate_bsm_to_file` tool to calculate prices.\n- Use the `calculate_